## MedMCQ dataset

In [10]:
from datasets import Dataset
dataset = Dataset.load_from_disk("/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final")
print(dataset)
print(dataset[0])

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 7819
})
{'id': 'f79a972f-3611-4cdc-837e-ce2f9393f592', 'question': 'Hyper viscosity is seen in', 'opa': 'Cryoglobulinemia', 'opb': 'Multiple myeloma', 'opc': 'MGUS', 'opd': 'Lymphoma', 'cop': 0, 'choice_type': 'single', 'exp': 'Ref William hematology 6/e p1268 The term cryoglobulinemia refers to the presence in the serum of proteins that precipitate at temperatures below 37 degrees C and redissolve on rewarming. ... The elective treatment for hyperviscosity syndrome, whether associated with monoclonal, mixed, or polyclonalcryoglobulinemia, is plasma exchange.', 'subject_name': 'Anatomy', 'topic_name': 'General anatomy'}


In [11]:
# throw 'cop' == -1 data
dataset = dataset.filter(lambda example: example['cop'] != -1)
print(dataset)
print(dataset[0])

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 7576
})
{'id': 'f79a972f-3611-4cdc-837e-ce2f9393f592', 'question': 'Hyper viscosity is seen in', 'opa': 'Cryoglobulinemia', 'opb': 'Multiple myeloma', 'opc': 'MGUS', 'opd': 'Lymphoma', 'cop': 0, 'choice_type': 'single', 'exp': 'Ref William hematology 6/e p1268 The term cryoglobulinemia refers to the presence in the serum of proteins that precipitate at temperatures below 37 degrees C and redissolve on rewarming. ... The elective treatment for hyperviscosity syndrome, whether associated with monoclonal, mixed, or polyclonalcryoglobulinemia, is plasma exchange.', 'subject_name': 'Anatomy', 'topic_name': 'General anatomy'}


In [12]:
from datasets import Dataset
import os
import json

root = "/projects/JHA/shared/dataset/medmcqa_diabetes_llm_final"

# Shuffle the dataset and select 1000 samples
sampled_dataset = dataset.shuffle(seed=42) #.select(range(1000))

# Function to format a single instance into a query
def format_query(instance):
    question = instance['question'].replace("\n", "")
    opa = instance['opa'].replace("\n", "")
    opb = instance['opb'].replace("\n", "")
    opc = instance['opc'].replace("\n", "")
    opd = instance['opd'].replace("\n", "")
    
    query = f"{question}: A. {opa}, B. {opb}, C. {opc}, D. {opd}. Please select the correct answer from A, B, C, D. Put your answer in \\boxed{{}}."
    return query

# Generate all queries
queries = [format_query(instance) for instance in sampled_dataset]

# Write all queries into a text file
output_file_path = os.path.join(root, "queries.txt")
with open(output_file_path, 'w') as file:
    file.write("\n".join(queries))

print(f"Queries have been written to {output_file_path}")


answers = [instance['cop'] for instance in sampled_dataset]

json_file = os.path.join(root, "answers.json")
with open(json_file, "w") as file:
    json.dump(answers, file, indent=4)  # `indent=4` makes the JSON file human-readable
print(f"answers have been written to {json_file}")

Queries have been written to /projects/JHA/shared/dataset/medmcqa_diabetes_llm_final/queries.txt
answers have been written to /projects/JHA/shared/dataset/medmcqa_diabetes_llm_final/answers.json


In [13]:
with open(output_file_path, "r") as file:
    queries = file.readlines()

for i, query in enumerate(queries):
    if 'A' not in query:
        print(f'{i}: {query}')

In [14]:
print(queries[6407])

Acute anginal attack can be terminated by: September 2007: A. Nitroglycerin, B. Digoxin, C. Lignocaine, D. Verapamil. Please select the correct answer from A, B, C, D. Put your answer in \boxed{}.



In [15]:
len(queries)

7576